**Парсинг дат**

In [4]:
import numpy as np
import os
import nltk
import re
import pickle
from time import time
from gensim import corpora, models, similarities
from itertools import chain
from _datetime import datetime
from lxml import etree
import json
# from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from math import log
from pprint import pprint as pp
from string import punctuation
from collections import defaultdict, deque, OrderedDict
import pandas as pd
import numpy
import logging

logging.basicConfig(format='%(levelname)s : %(message)s', level=logging.INFO)
logging.root.level = logging.INFO  # ipython sometimes messes up the logging setup; restore

C:\Anaconda3\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [5]:
from datetime import datetime

def parse_date(date_string, product_list=False):
    # date_only = date_string[:10]
    # '2009-01-01 08:25:07'
    # 2012-01-17T17:57:00-05:00
    # Friday November 10, 2006 10:49 pm PST
    # November 2, 2012 - product list
    if not product_list:
        if 'T' not in date_string:
            date = datetime.strptime(date_string, '%Y-%m-%d %H:%M:%S').date()
        elif 'pst' in date_string.lower() or 'pdt' in date_string.lower():
            date = datetime.strptime(date_string[:-4], '%A %B %d, %Y %H:%M %p').date()
        else:
            date = datetime.strptime(date_string[:-6], '%Y-%m-%dT%H:%M:%S').date() 
    else:
        date = datetime.strptime(date_string, '%B %d, %Y').date()
    return date
date = parse_date('November 2, 2012', True)
article_date = parse_date('Friday March 10, 2016 10:49 pm PST')

print(date, article_date, (article_date - date).days)
def parse_dates(dates):
    return [parse_date(date) for date in dates]

2012-11-02 2016-03-10 1224


**Препроцессинг**

In [6]:
def read_data(filename):
    """Reads the given file and creates a generator object returning one line at a time split by tabulation"""
    data = pd.read_excel(filename)
    return data

with open('..\english_stopwords.txt', 'r', encoding='utf-8') as f:
    stopwords = [word.strip() for word in f.readlines()]

lemmatizer = WordNetLemmatizer()
numbers = re.compile('[0-9]')
punctuation += '\n—–- «»\'\"'
lemmatization_stoplist = ['3gs', '4s', '5s', '6s']
product_lemm_reg = re.compile("(iphone|ipad|ipod|macbook|[ie]mac|air|5c|classic|nano|tv|ibook|shuffle|touch|mini|pro|powerbook)(s|es)")

def lemmatize(text, allow_stopwords=False):
    if allow_stopwords:
        current_stopwords = []
    else:
        current_stopwords = stopwords[:]
    lemmatized = [lemmatizer.lemmatize(word).lower() 
                  if not word in lemmatization_stoplist else word.lower() 
                  for word in nltk.word_tokenize(text) 
                    if word not in current_stopwords + list(punctuation)
                    and word not in ["''", ' ', '``', '', "'s"]
                 ]
    product_lemmatized = [product_lemm_reg.sub('\\1', word) for word in lemmatized]
    return product_lemmatized

def preprocess_text(text, split=None, allow_stopwords=False):
    if split == 'sentence':
        sentecnes = nltk.sent_tokenize(text)
        return [lemmatize(sent, allow_stopwords) for sent in elements]
    elif split == 'paragraph_sentence':
        try:
            paragraphs = [nltk.sent_tokenize(par) for par in re.split('[\r\n]{2,}', text) if not par.strip() == '']
        except:
            print(text)
        return [[lemmatize(sent, allow_stopwords) for sent in sents if not sent.strip() == ''] for sents in paragraphs]
    elif split == 'paragraph':
        paragraphs = [lemmatize(par, allow_stopwords) for par in re.split('[\r\n]{2,}', text)]
        return paragraphs
    else:
        return lemmatize(text)
    
def make_corpus(filename, serialize=None):
    data = read_data(filename)
    dates = parse_dates(data['datetime'])
    preprocessed_docs =  [[id, date, preprocess_text(body, split='paragraph_sentence')] for (id, body), date in zip(enumerate(list(data['body'])), dates) if not pd.isnull(body) and not body.strip() == '']
    if serialize:
        pickle_serialize(preprocessed_docs, serialize)
    return preprocessed_docs

def pickle_serialize(obj, filename):
    with open(filename, 'wb') as f:
        f.write(pickle.dumps(obj))

def load_pickle(filename):
    with open(filename, 'rb') as f:
        return pickle.loads(f.read())
    
# test_corpus = make_corpus('../DATA/test_data.xlsx')    

Загрузим и предобработаем все корпуса. 
Затем склеим их в один большой корпус, чтобы построить по нему тематическую модель.

In [7]:
# app_ins_corpus = make_corpus('../DATA/AppleInsider.xlsx', serialize = '../DATA/AppleInsiderPreprocessed.pickle')
# app_ins_corpus = [elt for elt in app_ins_corpus if elt[1].year >= 2009]
# pickle_serialize(app_ins_corpus, '../DATA/AppleInsiderPreprocessed.pickle')
app_ins_corpus = load_pickle('../DATA/AppleInsiderPreprocessed.pickle')
# app_ins_corpus[:10]

In [65]:
app_ins_corpus[0]

[0,
 datetime.date(2009, 1, 1),
 [[['addressing',
    'complaint',
    'nearly',
    'every',
    'iphone',
    'owner',
    'in',
    'northern',
    'climate',
    'apple',
    'ha',
    'filed',
    'patent',
    'would',
    'cover',
    'mean',
    'using',
    'multi-touch',
    'device',
    'with',
    'glove'],
   ['the',
    'us',
    'patent',
    'office',
    'application',
    'note',
    'capacitive',
    'touchcreen',
    'like',
    'on',
    'iphone',
    'ipod',
    'touch',
    'are',
    'problematic',
    'in',
    'colder',
    'weather'],
   ['as',
    'depend',
    'on',
    'electrical',
    'response',
    'from',
    'user',
    'fingertip',
    'is',
    'often',
    'blocked',
    'when',
    'wearing',
    'insulated',
    'glove',
    'screen',
    'either',
    'force',
    'user',
    'to',
    'take',
    'glove',
    'else',
    'sit',
    'tight',
    'return',
    'indoors']],
  [['apple',
    'solution',
    'would',
    'give',
    'glove',
    '

In [8]:
# mcr_corpus = make_corpus('../DATA/MacRummors.xlsx', serialize = '../DATA/MacRummorsPreprocessed.pickle')
# mcr_corpus = [elt for elt in mcr_corpus if elt[1].year >= 2009]
# pickle_serialize(mcr_corpus, '../DATA/MacRummorsPreprocessed.pickle')
mcr_corpus = load_pickle('../DATA/MacRummorsPreprocessed.pickle')
# mcr_corpus[:10]

In [9]:
# nfm_corpus = make_corpus('../DATA/NineToFiveMac.xlsx', serialize = '../DATA/NineToFiveMacPreprocessed.pickle')
nfm_corpus = load_pickle('../DATA/NineToFiveMacPreprocessed.pickle')

In [77]:
combined_corpus = defaultdict(list)
[combined_corpus[date].append(pars) for id, date, pars in app_ins_corpus]
[combined_corpus[date].append(pars) for id, date, pars in mcr_corpus]
[combined_corpus[date].append(pars) for id, date, pars in nfm_corpus]
cc = []
for date, docs in sorted(combined_corpus.items(), key=lambda x: x[0]):
    for pars in docs:
        for par in pars:
            cc.append(list(chain(*par)))

# combined_corpus = [[id, date, pars] for id, (date, pars) in enumerate(sorted(combined_corpus.items(), key=lambda x: x[0]))]
# combined_doc_corpus = defaultdict(list)
# [combined_doc_corpus[date].append(doc) for id, date, doc in app_ins_corpus]
# [combined_doc_corpus[date].append(doc) for id, date, doc in mcr_corpus]
# [combined_doc_corpus[date].append(doc) for id, date, doc in nfm_corpus]
# cc_docs = []
# id = 0
# for date, docs in sorted(combined_doc_corpus.items(), key=lambda x: x[0]):
#     for doc in docs:
#         cc_docs.append([id, date, doc])
#         id += 1
# cc_docs[:10]
# [[id, date, doc for doc in docs] for id, (date, docs) in enumerate(sorted(combined_doc_corpus.items(), key=lambda x: x[0]))]
# del(app_ins_corpus, mcr_corpus, nfm_corpus)

In [85]:
import datetime
cc[23002]

['as',
 'previously',
 'known',
 'iphone',
 'will',
 'be',
 'available',
 'in',
 'june',
 'exact',
 'date',
 'ha',
 'been',
 'announced',
 'the',
 'document',
 'also',
 'reinforces',
 '499',
 '599',
 'price',
 'iphone',
 'instructs',
 'employee',
 'to',
 'speculate',
 'discus',
 'pricing',
 'possibilties.related',
 'roundup',
 'iphone',
 '6s',
 'buyer',
 'guide',
 'iphone',
 'neutral']

**Подготавливаем данные для построения тематической модели**

In [11]:
from itertools import chain

def make_batches(corpus ,test=False):
    if test:
        first_batch_len = 100
    else:
        first_batch_len = 2000
    batch_id = 0
    batches = []
    prev_date = corpus[0][1]
    current_batch = []
    for id, date, paragraphs in corpus:
        if batch_id != 0:
            current_batch.extend([list(chain(*sents)) for sents in paragraphs])
            if len(current_batch) >= 30:
                batches.append([date, current_batch])
                current_batch = []
        else:
            if len(current_batch) < first_batch_len:
                current_batch.extend([list(chain(*sents)) for sents in paragraphs])
            else:
                prev_date = date
                batch_id += 1
    return batches

def make_dictionary(texts, test=False):
    dictionary = corpora.Dictionary(texts)
    if test:
        dictionary.filter_extremes(no_below=1, no_above=1000, keep_n=None)
    else:
        dictionary.filter_extremes(no_below=20, no_above=0.5, keep_n=None)
    print(dictionary)
    return  dictionary

def make_bow_texts(batch, dictionary):
    bow_texts = [dictionary.doc2bow(text) for text in batch]
    return bow_texts

# test_batches = make_batches(test_corpus, test=True)
# test_texts = [list(chain(*chain(*doc[2]))) for doc in test_corpus]
# test_dict = make_dictionary(test_texts, test=True) 
# test_bows = [[date, make_bow_texts(batch, test_dict)] for (date, batch) in test_batches]

In [12]:
cc_batches = make_batches(combined_corpus) # объединенный корпус, разбитый на батчи с документами за каждый день
cc_texts = [list(chain(*chain(*doc[2]))) for doc in combined_corpus] # просто список всех текстов во всем корпусе
cc_dict = make_dictionary(cc_texts)  #  словарь id2word слов на всем корпусе с
cc_bows =  [[date, make_bow_texts(batch, cc_dict)] for date, batch in cc_batches]  # весь корпус с параграфами переведенными в формат bow

INFO : adding document #0 to Dictionary(0 unique tokens: [])
INFO : built Dictionary(181592 unique tokens: ['batter-by-batter', 'homegrown', 'june.we', 'codifying', 'users.verizon']...) from 5462 documents (total 13201508 corpus positions)
INFO : discarding 165640 tokens: [('crabb', 5), ('i', 2856), ('note', 3198), ('we', 3035), ('on', 5069), ('even', 3169), ('well', 3598), ('//www.macinstein.com/pressrelease.cfm', 1), ('rachel', 10), ('saraceno', 1)]...
INFO : keeping 15952 tokens which were in no less than 20 and no more than 2731 (=50.0%) documents
INFO : resulting dictionary: Dictionary(15952 unique tokens: ['shifted', 'varies', 'reaching', 'glossy', 'modifying']...)


Dictionary(15952 unique tokens: ['shifted', 'varies', 'reaching', 'glossy', 'modifying']...)


**Строим модель**

In [13]:
def perplexity(model, corpus):
    """Calculates perpelexity of the given model on a given corpus of bow texts"""
    corpus_length = 0
    log_likelihood = 0
    topic_profiles = model.state.get_lambda() / np.sum(model.state.get_lambda(), axis=1)[:, np.newaxis]
    for document in corpus:
        gamma, _ = model.inference([document])
        document_profile = gamma / np.sum(gamma)
        for term_id, term_count in document:
            corpus_length += term_count
            term_probability = np.dot(document_profile, topic_profiles[:, term_id])
            log_likelihood += term_count * log(term_probability)
    perplexity = np.exp(-log_likelihood / corpus_length)
    return perplexity

In [14]:
def make_batch_model(dictionary, bow_batches, num_topics=400, chunksize=500, passes=10, batch=True, name= 'topics_' + str(datetime.now()).replace(':', '-')[:19]):   
    start = datetime.now()
    model = models.ldamulticore.LdaMulticore(id2word=dictionary, 
                                                     num_topics=num_topics, 
                                                     workers=3,
                                                     chunksize=chunksize,  
                                                     passes=passes,
                                                     iterations = 200
                                                     )
    os.makedirs(os.path.join('.', name), exist_ok = True)
    n_batches = len(bow_batches)
    for i, (date, batch) in enumerate(bow_batches):
        if i and i % 100 == 0: 
            print("{} of {} batches processed".format(i, n_batches))
        model.update(batch)
        if i % 7 == 0:
            with open(os.path.join('.', name, str(date)), 'wb') as f:
                f.write(pickle.dumps(model))
    with open(os.path.join('.', name, str(date)), 'wb') as f:
                f.write(pickle.dumps(model))
    print('Evaluation time: {}'.format((datetime.now() - start) / 60))
#     print('Perplexity: {}'.format(perplexity(model, bow_batches)))

In [91]:
def make_model(dictionary, doc_bows, num_topics=400, chunksize=500, passes=10, batch=True, name= 'topics_' + str(datetime.now()).replace(':', '-')[:19]):   
    start = datetime.now()
    model = models.ldamulticore.LdaMulticore(doc_bows,
                                             id2word=dictionary, 
                                             num_topics=num_topics, 
                                             workers=3,
                                             chunksize=chunksize,  
                                             passes=passes,
                                             iterations = 100
                                             )
    os.makedirs(os.path.join('.', name), exist_ok = True)
    model.save(os.path.join('.', name, 'model'))
    print('Evaluation time: {}'.format((datetime.now() - start) / 60))
    return model
#     print('Perplexity: {}'.format(perplexity(model, bow_batches)))

In [87]:
cc_dict = make_dictionary(cc)
cc_bows = make_bow_texts(cc, cc_dict)

INFO : adding document #0 to Dictionary(0 unique tokens: [])
INFO : adding document #10000 to Dictionary(14025 unique tokens: ['shifted', 'slotted', 'reaching', 'afterdark.com', 'previously-rumored']...)
INFO : adding document #20000 to Dictionary(22293 unique tokens: ['shifted', 'slotted', 'reaching', 'afterdark.com', 'homegrown']...)
INFO : adding document #30000 to Dictionary(30734 unique tokens: ['shifted', 'slotted', 'reaching', 'afterdark.com', 'modifying']...)
INFO : adding document #40000 to Dictionary(39095 unique tokens: ['shifted', 'slotted', 'reaching', 'dynamically.our', 'afterdark.com']...)
INFO : adding document #50000 to Dictionary(47977 unique tokens: ['40.3', 'slotted', 'reaching', 'afterdark.com', '149.28']...)
INFO : adding document #60000 to Dictionary(55019 unique tokens: ['40.3', 'slotted', 'applicationfirewall', 'reaching', 'afterdark.com']...)
INFO : adding document #70000 to Dictionary(60428 unique tokens: ['40.3', 'slotted', 'applicationfirewall', 'reaching',

Dictionary(18363 unique tokens: ['shifted', 'varies', 'reaching', 'glossy', 'modifying']...)


In [88]:
cc_bows[1004]

[(1400, 1),
 (1832, 1),
 (1962, 1),
 (2067, 1),
 (3741, 1),
 (4842, 1),
 (5714, 1),
 (5988, 1),
 (6276, 2),
 (7816, 1),
 (8753, 1),
 (9105, 1),
 (9413, 1),
 (9514, 1),
 (10027, 2),
 (10205, 1),
 (10798, 1),
 (10882, 1),
 (10936, 1),
 (11153, 1),
 (12221, 1),
 (12687, 1),
 (13058, 1),
 (13551, 1),
 (13718, 1),
 (14115, 1),
 (14135, 1),
 (14597, 1),
 (14599, 1),
 (15079, 1),
 (15271, 1),
 (18084, 1)]

In [ ]:
from datetime import datetime
model = make_model(cc_dict, cc_bows, num_topics = 1000, chunksize = 100, passes = 10, name='all_model')

INFO : using symmetric alpha at 0.001
INFO : using symmetric eta at 5.445733267984534e-05
INFO : using serial LDA version on this node


In [12]:
# model = make_model(test_dict, test_bows, num_topics = 10, chunksize = 5, passes = 10, name = 'test_model')
# model
# pickle.loads(list(model.items())[1][1]).show_topics()
# alltime_model_dict = make_model(cc_dict, cc_bows, num_topics = 500, chunksize = 5, passes = 2, name='paragraphs_unconverged_model3')
# with open('../DATA/by_day_topic_model.pickle', 'wb') as f:
#     f.write(pickle.dumps(alltime_model_dict))

In [13]:
def load_topic_models(directory):
    from datetime import datetime
    models_dict = {}
    for filename in os.listdir(directory):
        date = datetime.strptime(filename, '%Y-%m-%d').date()
        models_dict[date] = os.path.join(directory, filename)
    return models_dict

models_dict = load_topic_models('test_model')
models_dict

{datetime.date(2009, 1, 6): 'test_model\\2009-01-06',
 datetime.date(2009, 1, 16): 'test_model\\2009-01-16',
 datetime.date(2009, 1, 19): 'test_model\\2009-01-19'}

In [27]:
def load_model(filename):
    with open(filename, 'rb') as f:
        return pickle.loads(f.read())


def get_topic_distributions(dictionary, data, model_dict):
    import datetime
    print('start gathering')
    models_dates = list(model_dict.keys())
#     print(sorted(models_dates))
    topic_by_date_counts = defaultdict(lambda: defaultdict(int))
    date_by_topic_counts = defaultdict(lambda: defaultdict(int))
    current_model_date = min(models_dates)
    model = load_model(model_dict[current_model_date])
    print('model loaded')
    for id, date, paragraphs in data:
#         print('-------------------------\ncurdate', date)
        if date <= max(models_dates) and date <= datetime.date(2009, 1, 27):
            cur_topics = defaultdict(int)
            min_model_date = min([d for d in models_dates if d >= date])
            if current_model_date < min_model_date:
                model = load_model(model_dict[min_model_date])
                current_model_date = min_model_date
                print(current_model_date)
#             print('making bow...')
            par_bows = [dictionary.doc2bow(list(chain(*par))) for par in paragraphs]
#             print('Done')
            for bow in par_bows:
#                 print('getting topics for bow')
                topics = model.get_document_topics(bow, minimum_probability=0.5)
#                 pp(topics)
                for topic, prob in sorted(topics, key=lambda  x: -x[1])[:5]:
                    topic_terms = '+'.join([w for w, p in model.show_topic(topic, 5)])
                    cur_topics[topic_terms] += 1  # ID топиков со временем меняются, поэтому используем слова в к-ве id
            if cur_topics:
                doc_topics = sorted(cur_topics.items(), key=lambda x: -x[1])[:5]
                print(doc_topics)
#                 print(doc_topics)
                for topic, prob in doc_topics:
                    topic_by_date_counts[date][topic] += 1
                    date_by_topic_counts[topic][date] += 1
            
    return topic_by_date_counts, date_by_topic_counts
        
tbd, dbt = get_topic_distributions(test_dict, test_corpus,  models_dict)
dbt

start gathering
model loaded
[('also+offering+easy+dvd+graphic', 1)]
[('ability+without+see+related+mean', 1)]
[('order+request+single+find+saved', 1)]
[('new+label+drm-free+store+million', 1)]
[('order+request+single+find+saved', 1)]
[('price+high+charge+three+window.adsbygoogle', 1)]
[('order+request+single+find+saved', 2), ('photo+version+movie+definition+different', 1)]
[('order+request+single+find+saved', 1), ("pound+latest+removing+kg+'09", 1)]
[('order+request+single+find+saved', 3)]
[('battery+to+macbook+model+new', 2), ('order+request+single+find+saved', 1), ('ability+without+see+related+mean', 1)]
[('price+high+charge+three+window.adsbygoogle', 1)]
2009-01-16
[('series+that+needed+billion+fifth', 6), ('release+before+world+change+appleinsider', 2), ('exploit+malware+sandboxing+software+attack', 2)]
[('series+that+needed+billion+fifth', 4)]
[('series+that+needed+billion+fifth', 4), ('pc+move+system+writing+rather', 2), ('security+microsoft+latest+address+windows', 1)]
[('serie

defaultdict(<function __main__.get_topic_distributions.<locals>.<lambda>>,
            {'ability+without+see+related+mean': defaultdict(int,
                         {datetime.date(2009, 1, 1): 1,
                          datetime.date(2009, 1, 6): 1}),
             'access+security+installed+feature+new': defaultdict(int,
                         {datetime.date(2009, 1, 9): 1}),
             'also+offering+easy+dvd+graphic': defaultdict(int,
                         {datetime.date(2009, 1, 1): 1}),
             'battery+to+macbook+model+new': defaultdict(int,
                         {datetime.date(2009, 1, 6): 1}),
             'exploit+malware+sandboxing+software+attack': defaultdict(int,
                         {datetime.date(2009, 1, 7): 1,
                          datetime.date(2009, 1, 9): 1}),
             'internet+microsoft+monopoly+software+enough': defaultdict(int,
                         {datetime.date(2009, 1, 19): 1}),
             "n't+however+against+up+give": defa

In [74]:
# %matplotlib inline
%matplotlib qt 
import matplotlib.pyplot as plt
from scipy.interpolate import spline
from matplotlib.lines import Line2D
from itertools import product
linestyles = ['-', '--', ':']
markers = []
import random
for m in Line2D.markers:
    try:
        if len(m) == 1 and m != ' ':
            markers.append(m)
    except TypeError:
        pass

styles = markers + [
    r'$\lambda$',
    r'$\bowtie$',
    r'$\circlearrowleft$',
    r'$\clubsuit$',
    r'$\checkmark$']
colors = ('b', 'g', 'r', 'c', 'm', 'y', 'k')



def plot_topics(date_by_topic_counts, dates):
    line_styles = list(product(linestyles, colors))
    pp(line_styles)
    # line_styles
    datesecs =  [date.toordinal() for date in dates]
    width = 12
    height = 8
    plt.figure(figsize=(width, height))
    plt.ylim(0, 8)
    legend = []
    for topic in date_by_topic_counts:
        rand = random.randint(0, len(line_styles))
        line, color = line_styles.pop(rand)
#         line_styles.pop
        topic_counts = [date_by_topic_counts[topic].get(date, 0) for date in dates]
#         xnew = np.linspace(min(datesecs),max(datesecs), 5000)
#         topic_counts_sm = spline(datesecs ,topic_counts, xnew)
        plt.plot(dates, topic_counts, line, color=color, markersize=10)
#         plt.plot(xnew, topic_counts_sm)
        legend.append(topic[:50])
#         break
    plt.plot(dates, topic_counts)
    plt.legend(legend)
    plt.show()

dates = sorted(tbd.keys())
plot_topics(dbt, dates)

[('-', 'b'),
 ('-', 'g'),
 ('-', 'r'),
 ('-', 'c'),
 ('-', 'm'),
 ('-', 'y'),
 ('-', 'k'),
 ('--', 'b'),
 ('--', 'g'),
 ('--', 'r'),
 ('--', 'c'),
 ('--', 'm'),
 ('--', 'y'),
 ('--', 'k'),
 (':', 'b'),
 (':', 'g'),
 (':', 'r'),
 (':', 'c'),
 (':', 'm'),
 (':', 'y'),
 (':', 'k')]


IndexError: pop index out of range

In [29]:
cc_models_dict = load_topic_models('paragraphs_unconverged_model3')
cc_tbd, cc_dbt = get_topic_distributions(cc_dict, cc_docs,  cc_models_dict)

start gathering
model loaded
[('imac+nvidia+optical+box+aluminum', 1), ('inside+banner+black+together+height', 1)]
[('imac+nvidia+optical+box+aluminum', 2)]
[('multi-touch+response+return+often+else', 1), ('built-in+cellphone+output+suite+dropped', 1)]
[('multi-touch+response+return+often+else', 1), ('built-in+cellphone+output+suite+dropped', 1)]
[('ilife+imovie+charge+easily+entirely', 1)]
[('mark+november+net+december+holiday', 1)]
[('patent+filed+pcs+office+potentially', 1)]
[('patent+filed+pcs+office+potentially', 1)]
[('ilife+imovie+charge+easily+entirely', 1)]
[('unique+limit+expense+try+basic', 1)]
[('mark+november+net+december+holiday', 1)]
[('picture+perhaps+500+can’t+ain’t', 1), ('imac+nvidia+optical+box+aluminum', 1)]
[('intel+probably+individual+feed+...', 1)]
[('intel+probably+individual+feed+...', 1)]
[('beyond+outside+leave+capable+finger', 2), ('built-in+cellphone+output+suite+dropped', 1), ('mark+november+net+december+holiday', 1), ('iwork+seem+cloud+remain+native', 1)

In [30]:
cc_dbt

defaultdict(<function __main__.get_topic_distributions.<locals>.<lambda>>,
            {'2007+movie+rarely+accessory+imovie': defaultdict(int,
                         {datetime.date(2009, 1, 7): 2,
                          datetime.date(2009, 1, 11): 2,
                          datetime.date(2009, 1, 13): 5,
                          datetime.date(2009, 1, 15): 1,
                          datetime.date(2009, 1, 16): 2,
                          datetime.date(2009, 1, 25): 1}),
             'beyond+outside+leave+capable+finger': defaultdict(int,
                         {datetime.date(2009, 1, 4): 2,
                          datetime.date(2009, 1, 6): 1,
                          datetime.date(2009, 1, 9): 2,
                          datetime.date(2009, 1, 16): 2,
                          datetime.date(2009, 1, 20): 1,
                          datetime.date(2009, 1, 21): 4}),
             'built-in+cellphone+output+suite+dropped': defaultdict(int,
                         {datet

In [58]:
%matplotlib qt 
cc_dates = sorted(cc_tbd.keys())
plot_topics(cc_dbt, cc_dates)

IndexError: pop index out of range

**Считаем статистику по топикам в коллекции**

In [ ]:
def count_topics(paragrsph_split_texts, dictionary, threshold):
    """
    Counts the number of times each topic from a given model occurred in each of the bow_texts with corresponding 
    dates from the dates array.
    """
    date_counts = defaultdict(lambda: defaultdict(int))
    overall_counts = defaultdict(int)
    for id, date, paragraphs in paragrsph_split_texts:
        for paragraph in paragraphs:
            if type(paragraph) == list:
                bow = dictionary.doc2bow(list(chain(*paragraph)))
            else:    
                bow = dictionary.doc2bow(paragraph)
            if not len(paragraph) == 0:
                topics = model.get_document_topics(bow, minimum_probability=threshold)
                for topic, prob in topics:
                    date_counts[date][topic] += 1
                    overall_counts[topic] += 1
    return date_counts, sorted(overall_counts.items(), key=lambda x: -x[1])

def topics_words(model, sprted_topic_counts, topics_number=20, words_number=10):
    terms = []
    for topic, count in sprted_topic_counts[:topics_number]:
        most_frequent_words = model.show_topic(topic, words_number)
        terms.append([topic, count, ', '.join([word for word, prob in most_frequent_words])])
    return terms

test_date_counts, test_topic_counts = count_topics(test_corpus, test_dict, 0.5)
topics_words(model, test_topic_counts)-